In [1]:
from transformers import AutoTokenizer
from datasets import load_from_disk
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
billsum = load_from_disk("billsum")['train']
billsum = billsum.remove_columns(['Unnamed: 0', 'title'])

In [3]:
billsum = billsum.train_test_split(test_size=0.2)
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 15159
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 3790
    })
})

In [4]:
# transformer T5格式为:
# - single sequence: `X </s>`
# - pair of sequences: `A </s> B </s>`
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/root/miniconda3/envs/env_3812/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
prefix = "summarize: "


def preprocess_function(examples):
    # Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
# T5ForConditionalGeneration forward函数函数签名:
'''
def forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    decoder_input_ids: Optional[torch.LongTensor] = None,
    decoder_attention_mask: Optional[torch.BoolTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    decoder_head_mask: Optional[torch.FloatTensor] = None,
    cross_attn_head_mask: Optional[torch.Tensor] = None,
    encoder_outputs: Optional[Tuple[Tuple[torch.Tensor]]] = None,
    past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.FloatTensor], Seq2SeqLMOutput]:
'''
tokenized_billsum = billsum.map(preprocess_function, batched=True)
tokenized_billsum

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15159
    })
    test: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3790
    })
})

In [7]:
# 相当于torch.utils.data.DataLoader中collate_fn的作用(可以重写,参考K_demo/way_of_training/pytorch_transformer.ipynb)
# Data collator that will dynamically pad the inputs received, as well as the labels.
'''
model ([`PreTrainedModel`]):
    The model that is being trained. If set and has the *prepare_decoder_input_ids_from_labels*, use it to
    prepare the *decoder_input_ids*

    This is useful when using *label_smoothing* to avoid calculating loss twice.
'''
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [8]:
rouge = evaluate.load("rouge")
rouge.description

'ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for\nevaluating automatic summarization and machine translation software in natural language processing.\nThe metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.\n\nNote that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.\n\nThis metrics is a wrapper around Google Research reimplementation of ROUGE:\nhttps://github.com/google-research/google-research/tree/master/rouge\n'

In [9]:
def compute_metrics(eval_pred):
    # predictions.shape=[batch_size, max(该批次生成句子长度)]
    # labels.shape=[batch_size, max(该批次句子长度)]
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [10]:
# 继续预训练
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model_t5",
    save_total_limit=1,
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=4,

    # 区别于TrainingArguments特有参数:
    # predict_with_generate (bool, optional, defaults to False) — Whether to use generate to calculate generative metrics (ROUGE, BLEU).
    predict_with_generate=True
)

# 继承自Trainer
# predict方法:trainer.predict(tokenized_billsum["test"], **gen_kwargs)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# generate参数含义参考:huggingface GenerationConfig类
# 参考generate_help.md
trainer._gen_kwargs = {
    "repetition_penalty": 1.0,
    "max_length": 50,
    "min_length": 0}  # 用于评估

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/root/miniconda3/envs/env_3812/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 15159
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3792
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode th

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,2.355800,1.851220,0.224000,0.172900,0.213500,0.213500,18.998700
1000,2.014000,1.749843,0.229900,0.179900,0.219400,0.219400,19.000000
1500,1.927300,1.698408,0.234000,0.183700,0.223600,0.223600,19.000000
2000,1.900700,1.674968,0.235400,0.185300,0.225500,0.225500,19.000000
2500,1.851400,1.655808,0.237100,0.186800,0.227300,0.227300,19.000000
3000,1.850600,1.644527,0.237400,0.187300,0.227900,0.227900,19.000000
3500,1.829600,1.640118,0.237800,0.187800,0.228600,0.228600,19.000000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3790
  Batch size = 16
Saving model checkpoint to my_awesome_billsum_model_t5/checkpoint-500
Configuration saved in my_awesome_billsum_model_t5/checkpoint-500/config.json
Model weights saved in my_awesome_billsum_model_t5/checkpoint-500/pytorch_model.bin
tokenizer config file saved in my_awesome_billsum_model_t5/checkpoint-500/tokenizer_config.json
Special tokens file saved in my_awesome_billsum_model_t5/checkpoint-500/special_tokens_map.json
Copy vocab file to my_awesome_billsum_model_t5/checkpoint-500/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, summary. If

TrainOutput(global_step=3792, training_loss=1.9525265432108303, metrics={'train_runtime': 2358.7965, 'train_samples_per_second': 25.706, 'train_steps_per_second': 1.608, 'total_flos': 1.6413170948112384e+16, 'train_loss': 1.9525265432108303, 'epoch': 4.0})

In [12]:
# T5ForConditionalGeneration
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [16]:
# inference
inputs = tokenizer(
    "summarize: Shields a business entity from civil liability relating to any injury or death occurring at a facility of that entity in connection with a use of such facility by a nonprofit organization if: (1) the use occurs outside the scope of business of the business entity; (2) such injury or death occurs during a period that such facility is used by such organization; and (3) the business entity authorized the use of such facility by the organization. Makes this Act inapplicable to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including misconduct that: (1) constitutes a hate crime or a crime of violence or act of international terrorism for which the defendant has been convicted in any court; or (2) involves a sexual offense for which the defendant has been convicted in any court or misconduct for which the defendant has been found to have violated a Federal or State civil rights law. Preempts State laws to the extent that such laws are inconsistent with this Act, except State law that provides additional protection from liability. Specifies that this Act shall not be construed to supersede any Federal or State health or safety law. Makes this Act inapplicable to any civil action in a State court against a business entity in which all parties are citizens of the State if such State, citing this Act's authority and containing no other provision, enacts a statute declaring the State's election that this Act shall not apply to such action in the State.",
    return_tensors="pt").input_ids
inputs = inputs.to(model.device)
outputs = model.generate(inputs, repetition_penalty=1.0, min_length=0, max_length=50)
outputs

tensor([[    0,     3,    99,     3,     9,   268, 10409,    19,     3,  8860,
            21,   136,  2871,    42,  1687, 16198,    44,     3,     9,   268,
         10409,    16,  2135,    28,     3,     9,   169,    13,   224,  3064,
            57,     3,     9, 11069,  1470,     3,    99,    10,  5637,     8,
           169,  6986,  1067,     8,  7401,    13,   268,    13,     8,   268]],
       device='cuda:0')

In [17]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

if a business entity is liable for any injury or death occurring at a business entity in connection with a use of such facility by a nonprofit organization if: (1) the use occurs outside the scope of business of the business
